In [2]:
import os
openai_endpoint = os.getenv("OPENAI_ENDPOINT")
openai_apikey = os.getenv("OPENAI_APIKEY")
deployment_name = os.getenv("DEPLOYMENT_NAME")

In [3]:
from openai import AzureOpenAI

In [4]:
client = AzureOpenAI(api_key = openai_apikey, azure_endpoint = openai_endpoint,api_version = "2024-05-01-preview")

In [5]:
response = client.chat.completions.create(model = deployment_name,
                               messages=[{"role":"user","content":"Hi"}]
                              )

In [6]:
response.choices[0].message.content

'Hello! How can I help you today?'

code to validate if response is in required format

In [7]:
from dataclasses import dataclass
from typing import Any, Dict
import json
@dataclass
class Weather:
    Day: str
    Temperature_celcius: float
    Place: str


def validate_weather_input(data: Dict[str, Any]) -> Weather:
    required_fields = {'Day': str, 'Temperature_celcius': float, 'Place':str}
    for field, field_type in required_fields.items():
        if field not in data:
            raise ValueError(f"Missing field: {field}")
        if not isinstance(data[field], field_type):
            raise TypeError(f"Incorrect type for field {field}: expected {field_type}, got {type(data[field])}")
    return Weather(**data)

with no reference example, outformat fails in validation step

In [8]:
prompt = """For below given content, give Json response with Day, Temperature in celcius and Place as key-value pairs.
CONTENT: With 1 degree celcius London witnessed coldest day of the week on Thursday.
"""
response = client.chat.completions.create(model = deployment_name,
                               messages=[{"role": "system","content": "You're an helpful AI assistant"},
                                         {"role":"user","content":prompt}]
                              )
json_response = response.choices[0].message.content
print(json_response)
validate_weather_input(json.loads(json_response))

{
  "Day": "Thursday",
  "Temperature": 1,
  "Place": "London"
}


ValueError: Missing field: Temperature_celcius

with valid output format given in instruction, it might still fail at output type

In [9]:
prompt = """For below given content, give Json response with Day, Temperature in celcius and Place as key-value pairs.
CONTENT: With 1 degree celcius London witnessed coldest day of the week on Thursday.
RESPONSE FORMAT:
{
Day : <day>,
Temperature_celcius: <temperature>,
Place : <place>
}
"""
response = client.chat.completions.create(model = deployment_name,
                               messages=[{"role": "system","content": "You're an helpful AI assistant"},
                                         {"role":"user","content":prompt}]
                              )
json_response=response.choices[0].message.content
print(json_response)
validate_weather_input(json.loads(json_response))

{
  "Day": "Thursday",
  "Temperature_celcius": 1,
  "Place": "London"
}


TypeError: Incorrect type for field Temperature_celcius: expected <class 'float'>, got <class 'int'>

pydantic can be used to validate most responses, also openai in backend is integrated with pydantic for reposnse formats validations

In [10]:
!pip install -qq pydantic

In [11]:
from pydantic import BaseModel
class Weather(BaseModel):
    Day:str
    Temperature_celcius:float
    Place:str

above case where normal output validation failed due to type check is now validated with pydantic approach

In [12]:
Weather.validate(json.loads(json_response))

Weather(Day='Thursday', Temperature_celcius=1.0, Place='London')

one more example of normal validation

In [13]:
prompt = """For below given content, give Json response with Day, Temperature in celcius and Place as key-value pairs.
CONTENT: With 30 Fahrenheit London witnessed coldest day of the week on Thursday.
RESPONSE FORMAT:
{
Day : <day>,
Temperature_celcius: <temperature>,
Place : <place>
}
"""
response = client.chat.completions.create(model = deployment_name,
                               messages=[{"role": "system","content": "You're an helpful AI assistant"},
                                         {"role":"user","content":prompt}]
                              )
json_response=response.choices[0].message.content
print(json_response)
validate_weather_input(json.loads(json_response))

{
  "Day": "Thursday",
  "Temperature_celcius": "-1.1",
  "Place": "London"
}


TypeError: Incorrect type for field Temperature_celcius: expected <class 'float'>, got <class 'str'>

validation with pydantic approach

In [14]:
Weather.validate(json.loads(json_response))

Weather(Day='Thursday', Temperature_celcius=-1.1, Place='London')

# Function calling for given input is one of the feature ffrom OpenAI, through which cases where arguments needs to be created to call function analysing given input can be acheived


well defined simple function weather_status with parameters day,temeprature and place

In [15]:
def weather_status(day:str,temperature:float,place:str):
    """
    gives weather details of given place for given day

    Parameters:
    day(str) : day of the week
    temperature(float): temperature in celcius
    place(str): place name for given weather details

    Returns:
    dict: a dictionary with weather details
    """
    weather_details = {
        "day":day,
        "temperature":temperature,
        "place":place
    }
    return weather_details
    

well defined simple function weather_alert with parameters day,temeprature, place and supervisor

In [16]:
def weather_alert(day:str,temperature:float,place:str,supervisor:str):
    """
    gives weather details to create alret to supervisor of given place for given day

    Parameters:
    day(str) : day of the week
    temperature(float): temperature in celcius
    place(str): place name for given weather details
    supervisor(str): supervisor name for given place

    Returns:
    dict: a dictionary with weather details along with supervisor name
    """
    weather_details = {
        "day":day,
        "temperature":temperature,
        "place":place,
        "supervisor":supervisor
    }
    return weather_details

creating detailed schema of functions to be called

In [17]:
weather_status_function_details = {
        "type":"function",
        "function":{
        "name": "weather_status",  # Function Name
        "description": "gives weather details of given place for given day",  # Meta information of function; be as detailed as possible
        "parameters": {  # in json schema
            "type": "object",
            "properties": {
                "day": {
                    "type": "string",
                    "description": "day of the week",
                },
                "temperature": {
                    "type": "number",
                    "description": "temperature in celcius",
                },
                "place": {
                    "type": "string",
                    "description": "place name for given weather details",
                },
            },
            "required": ["day", "temperature", "place"],
        },
        }
    }

In [18]:
weather_alert_function_details = {
        "type":"function",
        "function":{
        "name": "weather_alert",
        "description": "gives weather details to create alret to supervisor of given place for given day",
        "parameters": {
            "type": "object",
            "properties": {
                "day": {
                    "type": "string",
                    "description": "day of the week",
                },
                "temperature": {
                    "type": "number",
                    "description": "temperature in celcius",
                },
                "place": {
                    "type": "string",
                    "description": "place name for given weather details",
                },
                "supervisor": {
                    "type": "string",
                    "description": "supervisor name for given place"
                }
            },
            "required": ["day", "temperature", "place","supervisor"],
        },
        }
    }

In [19]:
function_list = [weather_status_function_details,weather_alert_function_details]


functions parameter is now depreacted and tools should be used instead

In [20]:
response = client.chat.completions.create(model = deployment_name,
                               messages=[{"role": "system","content": "You're an helpful AI assistant"},
                                         {"role":"user","content":"People are so afraid of the current situation in Delhi and are vacating city, as it's reported with 270 kelvin unit temperature on Monday, the least in this year"}],
                               tools = function_list,
                              )

since given prompt covers case of function calling, no message response

In [21]:
print(response.choices[0].message.content)

None


with given instruction function weather_status needs to be invoked, so below outputs

In [22]:
response.choices[0].message.tool_calls[0].function.arguments

'{"day":"Monday","temperature":270,"place":"Delhi"}'

In [23]:
response.choices[0].message.tool_calls[0].function.name

'weather_status'

In [24]:
response = client.chat.completions.create(model = deployment_name,
                               messages=[{"role": "system","content": "You're an helpful AI assistant"},
                                         {"role":"user","content":"People are so afraid of the current situation in Delhi and are vacating city, as it's reported with 270 kelvin unit temperature on Monday. A weather is needed to Ganesh the location supervisor."}],
                               tools = function_list,
                              )

with given instruction function weather_alert needs to be invoked, so below outputs

In [25]:
response.choices[0].message.tool_calls[0].function.arguments

'{"day":"Monday","temperature":270,"place":"Delhi","supervisor":"Ganesh"}'

In [26]:
response.choices[0].message.tool_calls[0].function.name

'weather_alert'

pydantic can be used to create schema of class object, whose response can be adjusted as per requirement of function calling schema, to avoid writing schema as repeated tasks

In [27]:
from pydantic import BaseModel,Field

In [28]:
class Weather_details(BaseModel):
    "this is to get weather details of given place for given day"
    day:str = Field(description = "day of the week")
    temperature:float = Field(description = "temperature in celcius")
    place:str = Field(description = "place name for given weather details")

In [29]:
 Weather_details.model_json_schema()

{'description': 'this is to get weather details of given place for given day',
 'properties': {'day': {'description': 'day of the week',
   'title': 'Day',
   'type': 'string'},
  'temperature': {'description': 'temperature in celcius',
   'title': 'Temperature',
   'type': 'number'},
  'place': {'description': 'place name for given weather details',
   'title': 'Place',
   'type': 'string'}},
 'required': ['day', 'temperature', 'place'],
 'title': 'Weather_details',
 'type': 'object'}